In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
! pip install -q kaggle

In [ ]:
! mkdir -p ~/.kaggle

In [ ]:
from google.colab import files

In [ ]:
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
! cp kaggle.json ~/.kaggle

In [ ]:
! chmod 600 /root/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d tongpython/cat-and-dog

 99% 215M/218M [00:01<00:00, 186MB/s]
100% 218M/218M [00:01<00:00, 168MB/s]


In [ ]:
! unzip -q /content/cat-and-dog.zip

In [ ]:
training_dir = '/content/training_set/training_set'
test_dir = '/content/test_set/test_set'

# Building the model

In [ ]:
img_shape =(128,128,3)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape= img_shape, include_top=False, weights= 'imagenet')

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][

In [ ]:
base_model.trainable = False

In [ ]:
base_model.output

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [ ]:
global_average_layer

<KerasTensor: shape=(None, 1280) dtype=float32 (created by layer 'global_average_pooling2d')>

In [ ]:
prediction_layer = tf.keras.layers.Dense(units=1,activation ='sigmoid')(global_average_layer)

# Define Transfer Learning Model

In [ ]:
model= tf.keras.models.Model(inputs=base_model.input, outputs = prediction_layer)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 64, 64, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 64, 64, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 64, 64, 32)   0           ['bn_Conv1[0][0]']           

In [ ]:
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

In [ ]:
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_generator_train = ImageDataGenerator(rescale=1/255.0)
data_generator_test = ImageDataGenerator(rescale=1/255.0)

In [ ]:
train_gen = data_generator_train.flow_from_directory(directory=training_dir, target_size=(128,128), batch_size = 128, class_mode='binary')
test_gen = data_generator_train.flow_from_directory(directory=test_dir, target_size=(128,128), batch_size = 128, class_mode='binary')

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


# Training the Model

In [ ]:
model.fit_generator(generator=train_gen, epochs=5, validation_data=test_gen)

<ipython-input-27-da3e2306c20a>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=train_gen, epochs=5, validation_data=test_gen)


Epoch 1/5
63/63 [==============================] - 101s 2s/step - loss: 0.5229 - accuracy: 0.7493 - val_loss: 0.4056 - val_accuracy: 0.8403
Epoch 2/5
63/63 [==============================] - 102s 2s/step - loss: 0.3286 - accuracy: 0.8809 - val_loss: 0.2723 - val_accuracy: 0.9026
Epoch 3/5
63/63 [==============================] - 95s 2s/step - loss: 0.2369 - accuracy: 0.9177 - val_loss: 0.2063 - val_accuracy: 0.9278
Epoch 4/5
63/63 [==============================] - 92s 1s/step - loss: 0.1885 - accuracy: 0.9333 - val_loss: 0.1678 - val_accuracy: 0.9402
Epoch 5/5
63/63 [==============================] - 97s 2s/step - loss: 0.1613 - accuracy: 0.9428 - val_loss: 0.1459 - val_accuracy: 0.9471
